In [1]:
import tweepy
import pandas as pd
from textblob import TextBlob
from datetime import datetime
import time
import ast
from os import listdir
from os.path import isfile, join
import matplotlib.pyplot as plt
from wordcloud import WordCloud, STOPWORDS
import sys
sys.path.insert(1, 'C:\\Users\\raide\\OneDrive\\Documents\\GitHub\\capstone_project\\scraping')
sys.path.insert(1, 'C:\\Users\\raide\\OneDrive\\Documents\\GitHub\\capstone_project\\constants')
from scrape_hashtags import get_hashtag_stats
from constants import get_matteo_twitter_creds, get_michael_twitter_creds

access_token, access_token_secret, consumer_key, consumer_secret = get_michael_twitter_creds()

## Get hashtag data
Ideally, will have a way to get important hashtags, evaluate their exposure/popularity, and then feed it into the scraping function.

In [2]:
# Capture food words from https://www.enchantedlearning.com/wordlist/food.shtml
df = get_hashtag_stats()
df

,hashtag,unique_tweets_per_hour,retweets_per_hour,views_per_hour
0,apple,167,125,6161321
1,avocado,4,4,3750
2,bacon,4,13,2221
3,bake,21,0,5238
4,banana,8,21,950
...,...,...,...,...
460,watermelon,4,0,8
461,whey,4,0,8062
462,wok,0,4,0
463,yolk,0,4,0


In [6]:
df[['unique_tweets_per_hour', 'retweets_per_hour', 'views_per_hour']] = df[['unique_tweets_per_hour', 'retweets_per_hour', 'views_per_hour']].apply(pd.to_numeric)

In [7]:
all_words = df.hashtag.tolist()
all_hashtags = ['#' + s for s in all_words]

In [10]:
exposed_hashtags = df[df.views_per_hour >= 100000]
exposed_hashtags_words = [s for s in exposed_hashtags.hashtag.tolist()]
exposed_hashtags_hashtags = ['#' + s for s in exposed_hashtags_words]
exposed_hashtags_and_words = exposed_hashtags_words + exposed_hashtags_hashtags
print(f"Out of the {df.shape[0]} hashtags in the raw data, {exposed_hashtags.shape[0]} received >= 100000 views.")

Out of the 465 hashtags in the raw data, 29 received >= 100000 views.


In [11]:
class HandleExceptions(tweepy.Stream):
    
    def on_connection_error(self):
        self.disconnect()
        print(f'Stream has disconnected.\nNumber of tweets streamed: {self.captured_tweets}]\nNumber of tweets missed: {self.missed_tweets}\nPercent of tweets streamed that were captured: {self.missed_tweets / self.captured_tweets * 100}')
        return self.df
        
    def on_closed(self, response):
        print('Response', response)
        self.disconnect()
        print(f'Stream has disconnected.\nNumber of tweets streamed: {self.captured_tweets}]\nNumber of tweets missed: {self.missed_tweets}\nPercent of tweets streamed that were captured: {self.missed_tweets / self.captured_tweets * 100}')
        return self.df
    
    def on_disconnect(self):
        self.disconnect()
        print(f'Stream has disconnected.\nNumber of tweets streamed: {self.captured_tweets}]\nNumber of tweets missed: {self.missed_tweets}\nPercent of tweets streamed that were captured: {self.missed_tweets / self.captured_tweets * 100}')
        return self.df
    
    def on_exception(self, exception):
        print('An exception occurred:', exception)
        self.disconnect()
        print(f'Stream has disconnected.\nNumber of tweets streamed: {self.captured_tweets}]\nNumber of tweets missed: {self.missed_tweets}\nPercent of tweets streamed that were captured: {self.missed_tweets / self.captured_tweets * 100}')
        return self.df
        
    def on_request_error(self, status_code):
        print('An error occurred:', status_code)
        self.disconnect()
        print(f'Stream has disconnected.\nNumber of tweets streamed: {self.captured_tweets}]\nNumber of tweets missed: {self.missed_tweets}\nPercent of tweets streamed that were captured: {self.missed_tweets / self.captured_tweets * 100}')
        return self.df

class FoodScraper(tweepy.Stream):
    
    def on_connect(self):
        self.df = pd.DataFrame(columns = [  'created_at',
                                            'tweet_id',
                                            'user_id',
                                            'user_name',
                                            'screen_name',
                                            'verified',
                                            'text',
                                            'quote_tweet',
                                            'rewteet_count',
                                            'favorite_count',
                                            'place',
                                            'quote_status_id',
                                            'hashtags'])

    def on_status(self, status):
        self.captured_tweets = 0
        created_at = status.created_at
        tweet_id = status.id
        user_id = status.user.id
        user_name = status.user.name
        screen_name = status.user.screen_name
        verified = status.user.verified
        text = status.text
        quote_tweet = status.is_quote_status
        rewteet_count = status.retweet_count
        favorite_count = status.favorite_count
        
        try:
            place = status.place.bounding_box
            quote_status_id = status.quoted_status_id
            hashtags = status.entities.hashtags
        except:
            place = None
            quote_status_id = None
            hashtags = None
            
        self.tweets = [ created_at,
                        tweet_id,
                        user_id,
                        user_name,
                        screen_name,
                        verified,
                        text,
                        quote_tweet,
                        rewteet_count,
                        favorite_count,
                        place,
                        quote_status_id,
                        hashtags]
        
        self.captured_tweets += 1
        self.df.loc[len(self.df)] = self.tweets
        print(self.tweets)
    
    def on_limit(self, track):
        self.missed_tweets = 0
        self.missed_tweets += track

    def exceptions(self):
        HandleExceptions(self)

In [7]:
food_scraper = FoodScraper(consumer_key, consumer_secret, access_token, access_token_secret, max_retries=10)
        
try:
    df = food_scraper.filter(track=exposed_hashtags_and_words, languages=['en'])
except:
    food_scraper.disconnect()
    time.sleep(60)
    
df

[datetime.datetime(2021, 10, 16, 19, 44, 22, tzinfo=datetime.timezone.utc), 1449461232126373909, 1869812658, 'stayW ~gettr *', 'StayTelavi', False, "RT @RealPNavarro: Fire Fauci Video ad debuts on John Fredericks and War Room next week.  Here's your sneak peek.  FULL REPARATIONS FROM CCP…", False, 0, 0, None, None, None]
[datetime.datetime(2021, 10, 16, 19, 44, 22, tzinfo=datetime.timezone.utc), 1449461232893775872, 872557380705239040, 'JK 🦇', 'JK_9505', False, 'How appropriate that today\'s word was "candy" just after the choco chicken thing, the irony 🤣', False, 0, 0, None, None, None]
[datetime.datetime(2021, 10, 16, 19, 44, 22, tzinfo=datetime.timezone.utc), 1449461233082589188, 1013637254726766592, 'roses', 'kairalet13k', False, 'RT @SGChartUpdate: “Let Somebody Go” by Selena Gomez and #Coldplay enters the Top 15 (#14) on the Apple Music Worldwide Chart. 🌎 \n\nhttps://…', False, 0, 0, None, None, None]
[datetime.datetime(2021, 10, 16, 19, 44, 22, tzinfo=datetime.timezone.utc), 144

In [ ]:
df